In [ ]:
!apt-get update
!pip install pyspark findspark

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,679 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,692 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages 

# Configuration et init Spark

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
import json
import os

CONFIG_PATH = "/content/spark_config.json"

def init_spark():
    spark = SparkSession.builder.master("local[*]").getOrCreate()
    sc = spark.sparkContext

    # Sauvegarde de la configuration Spark
    config = spark.sparkContext.getConf().getAll()
    with open(CONFIG_PATH, "w") as f:
        json.dump(dict(config), f)

    print("Spark init")
    return spark, sc

def load_spark():
    if not os.path.exists(CONFIG_PATH):
        raise FileNotFoundError("Lancer d'abord `init_spark()`.")

    # Charger config
    with open(CONFIG_PATH, "r") as f:
        config = json.load(f)

    # Recréer session spark avec la même config
    spark_builder = SparkSession.builder.master("local[*]")
    for k, v in config.items():
        spark_builder = spark_builder.config(k, v)

    spark = spark_builder.getOrCreate()
    sc = spark.sparkContext
    print("Spark chargé avec la config sauvegardée")
    return spark, sc
